In [12]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/robinlinacre/Documents/data_linking/splink/')

## Historical people: Quick and dirty

This example shows how to get some initial record linkage results as quickly as possible.  

There are many ways to improve the accuracy of this model.  But this may be a good place to start if you just want to give Splink a try and see what it's capable of.

In [13]:
import pandas as pd 
df = pd.read_parquet("../data/historical_figures_with_errors_50k.parquet")
df = df.drop(["full_name", "first_and_surname"], axis=1)
df.head(5)

,unique_id,cluster,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


In [14]:
from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.duckdb import duckdb_comparison_library as cl
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.dob = r.dob",
        "l.postcode_fake = r.postcode_fake",
    ],
    "comparisons": [
        cl.levenshtein_at_thresholds("first_name", [1,3,5], term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("surname", [1,3,5], term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("dob", [1,2], term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("postcode_fake", 2),
        cl.exact_match("birth_place", term_frequency_adjustments=True),
        cl.exact_match("occupation",  term_frequency_adjustments=True),
    ],       
    
}

In [15]:
linker = DuckDBLinker(df, settings, set_up_basic_logging=False)
deterministic_rules = [
    "l.first_name = r.first_name and l.surname = r.surname",
    "l.first_name = r.first_name and l.dob = r.dob and l.birth_place = r.birth_place",
    "l.dob = r.dob",
    "l.postcode_fake = r.postcode_fake",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


In [16]:
linker.estimate_u_using_random_sampling(target_rows=1e7)

In [17]:
results = linker.predict(threshold_match_probability=0.9)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'surname':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained
Comparison: 'postcode_fake':
    m values not fully trained
Comparison: 'birth_place':
    m values not fully trained
Comparison: 'occupation':
    m values not fully trained


In [23]:
query = 'first_name_l != first_name_r or surname_l != surname_r '
results.as_pandas_dataframe(limit=100000).query(query).sample(1).T


,89185
match_weight,28.229723
match_probability,1.0
unique_id_l,Q2385895-5
unique_id_r,Q2385895-6
first_name_l,charlie
first_name_r,charles
gamma_first_name,2
surname_l,marshall
surname_r,marshall
gamma_surname,4
